### Frequent Itemsets and Association Rules

In [24]:
#Frequent Itemsets and Association Rules (Assignment #2)

#Import pyspark libraries and packages
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext
#uncomment next statement for multiple runs SparkContext method
#sc.stop()

In [2]:
sc = SparkContext()

In [3]:
import pandas as pd
data_xls = pd.read_excel('Data/Online Retail.xlsx', 'Online Retail', index_col=None)
type(data_xls)

pandas.core.frame.DataFrame

In [4]:
total_rows = len(data_xls)
print(total_rows)

541909


In [5]:
#data_xls
data_xls.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
#cast the grouping variable InvoiceNo and basket item StockCode variable to to a pandas string
data_xls['StockCode'] = data_xls['StockCode'].astype('str', copy=True, errors='raise')
data_xls['InvoiceNo'] = data_xls['InvoiceNo'].astype('str', copy=True, errors='raise')
#Remove invoice records that contain a 'C' since they are credits/returns
data_xls = data_xls[~data_xls['InvoiceNo'].str.contains('C')]

#view dataframe properties
data_xls.tail()
data_xls.dtypes


InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
#Get only invoice transactions from one country
df = data_xls.query('Country == "United Kingdom"')

#View properties
print('DF shape: ', df.shape)
total_rows = len(df)
print(total_rows)
type(df)

DF shape:  (487622, 8)
487622


pandas.core.frame.DataFrame

In [8]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [9]:
#Group records into unique invoice purchases
basket = df.groupby('InvoiceNo')['StockCode'].apply(','.join)
basket = basket.reset_index()
# values = ",".join(map(str, value_list)) 

#View resulting data
basket.head()

,InvoiceNo,StockCode
0,536365,"85123A,71053,84406B,84029G,84029E,22752,21730"
1,536366,"22633,22632"
2,536367,"84879,22745,22748,22749,22310,84969,22623,2262..."
3,536368,"22960,22913,22912,22914"
4,536369,21756


In [10]:
type(basket)


pandas.core.frame.DataFrame

In [11]:
#print(len(data))
newdf = basket[basket.columns[1]] 
newdf.head()

0        85123A,71053,84406B,84029G,84029E,22752,21730
1                                          22633,22632
2    84879,22745,22748,22749,22310,84969,22623,2262...
3                              22960,22913,22912,22914
4                                                21756
Name: StockCode, dtype: object

In [12]:
newdf.dtypes
type(newdf)
newdf =newdf.reset_index()
type(newdf)

#Save to text file so that spark could read in as RDD object type
newdf[:5].to_csv('Online Retail.txt', columns = ["StockCode"], header=False, index=False)

In [13]:
#The following statements were used to convert spark DF to spark RDD, though 
#read in as RDD directly from earlier saved text file
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlCtx = SQLContext(sc)
#transactions = rddlist.map(lambda line: line.strip().split(','))


In [14]:
#take in array and map to another array
data = sc.textFile("Online Retail.txt")
transactions = data.map(lambda line: line.strip().split(','))

type(transactions)
print(transactions.count())
#type(unique)
#print(unique.count())
transactions

5


PythonRDD[3] at RDD at PythonRDD.scala:48

In [15]:
#View result RDD data
for x in transactions.collect():
  print(x)

['"85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730"']
['"22633', '22632"']
['"84879', '22745', '22748', '22749', '22310', '84969', '22623', '22622', '21754', '21755', '21777', '48187"']
['"22960', '22913', '22912', '22914"']
['21756']


__Answer the following questions using FP-Growth__

1) How many frequent itemsets were found with minimum support of .15?

2) How many association rules were identified with a minimum confidence of .4?

3) What is the support and confidence of the following association rule: ['21754', '22748'] => ['22745']? Go beyond reporting the values, but include an interpretation of these evaluation metrics.

##### 1. How many frequent itemsets were found with minimum support of .15?

In [16]:
#FPGrowth.train returns an FP-Growth model that stores the frequent itemsets with their frequencies

model = FPGrowth.train(transactions, minSupport = 0.15, numPartitions = 10)
#type(model)
freqItemsets = model.freqItemsets().collect()

lf = len(freqItemsets)

print('Number of frequent itemsets with minimum support .15 :', lf,'\n')


Number of frequent itemsets with minimum support .15 : 4241 



In [17]:
#Print items in RDD returned
for item in freqItemsets:
    print(item)


FreqItemset(items=['22913'], freq=1)
FreqItemset(items=['22913', '22914"'], freq=1)
FreqItemset(items=['22913', '22912'], freq=1)
FreqItemset(items=['22913', '22912', '22914"'], freq=1)
FreqItemset(items=['22913', '22912', '"22960'], freq=1)
FreqItemset(items=['22913', '22912', '"22960', '22914"'], freq=1)
FreqItemset(items=['22913', '"22960'], freq=1)
FreqItemset(items=['22913', '"22960', '22914"'], freq=1)
FreqItemset(items=['22749'], freq=1)
FreqItemset(items=['22749', '22748'], freq=1)
FreqItemset(items=['22749', '22748', '"84879'], freq=1)
FreqItemset(items=['22749', '22748', '"84879', '21755'], freq=1)
FreqItemset(items=['22749', '22748', '22745'], freq=1)
FreqItemset(items=['22749', '22748', '22745', '"84879'], freq=1)
FreqItemset(items=['22749', '22748', '22745', '"84879', '21755'], freq=1)
FreqItemset(items=['22749', '22748', '22745', '21755'], freq=1)
FreqItemset(items=['22749', '22748', '48187"'], freq=1)
FreqItemset(items=['22749', '22748', '48187"', '"84879'], freq=1)
Freq

FreqItemset(items=['21754', '22310', '22622', '21777', '22748', '48187"', '21755'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22748', '21755'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748', '"84879'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748', '"84879', '21755'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748', '22745'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748', '22745', '"84879'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748', '22745', '"84879', '21755'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748', '22745', '21755'], freq=1)
FreqItemset(items=['21754', '22310', '22622', '21777', '22749', '22748', '48

In [19]:
# Print frequent itemsets

freqItemsetsDict = {}

print ("Frequent itemsets:")
for i in range(0, len(freqItemsets)):
    freqItemsetsDict[str(freqItemsets[i].items)] = freqItemsets[i].freq
    print ("\t%s" % freqItemsets[i].items)
    

Frequent itemsets:
	['22913']
	['22913', '22914"']
	['22913', '22912']
	['22913', '22912', '22914"']
	['22913', '22912', '"22960']
	['22913', '22912', '"22960', '22914"']
	['22913', '"22960']
	['22913', '"22960', '22914"']
	['22749']
	['22749', '22748']
	['22749', '22748', '"84879']
	['22749', '22748', '"84879', '21755']
	['22749', '22748', '22745']
	['22749', '22748', '22745', '"84879']
	['22749', '22748', '22745', '"84879', '21755']
	['22749', '22748', '22745', '21755']
	['22749', '22748', '48187"']
	['22749', '22748', '48187"', '"84879']
	['22749', '22748', '48187"', '"84879', '21755']
	['22749', '22748', '48187"', '22745']
	['22749', '22748', '48187"', '22745', '"84879']
	['22749', '22748', '48187"', '22745', '"84879', '21755']
	['22749', '22748', '48187"', '22745', '21755']
	['22749', '22748', '48187"', '21755']
	['22749', '22748', '21755']
	['22749', '"84879']
	['22749', '"84879', '21755']
	['22749', '22745']
	['22749', '22745', '"84879']
	['22749', '22745', '"84879', '21755']
	[

	['21754', '22310', '22623', '22622', '22749', '48187"']
	['21754', '22310', '22623', '22622', '22749', '48187"', '"84879']
	['21754', '22310', '22623', '22622', '22749', '48187"', '"84879', '21755']
	['21754', '22310', '22623', '22622', '22749', '48187"', '22745']
	['21754', '22310', '22623', '22622', '22749', '48187"', '22745', '"84879']
	['21754', '22310', '22623', '22622', '22749', '48187"', '22745', '"84879', '21755']
	['21754', '22310', '22623', '22622', '22749', '48187"', '22745', '21755']
	['21754', '22310', '22623', '22622', '22749', '48187"', '21755']
	['21754', '22310', '22623', '22622', '22749', '21755']
	['21754', '22310', '22623', '22622', '"84879']
	['21754', '22310', '22623', '22622', '"84879', '21755']
	['21754', '22310', '22623', '22622', '22745']
	['21754', '22310', '22623', '22622', '22745', '"84879']
	['21754', '22310', '22623', '22622', '22745', '"84879', '21755']
	['21754', '22310', '22623', '22622', '22745', '21755']
	['21754', '22310', '22623', '22622', '48187"

##### 2) How many association rules were identified with a minimum confidence of .4?

In [20]:
#Set minimum confidence (The number of times the association rule is found to be true)
minConfidence = .4
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), 
    key=lambda x: x.confidence(), reverse=True)
ar = len(rules)

print('The number of association rules identified with minimum confidence of 0.4 is:', ar,'\n')


The number of association rules identified with minimum confidence of 0.4 is: 25035 



In [21]:
#Print associations rules with support and confidence analysis variables 
for j in range(0, len(rules)):
    antecedent = list(rules[j].antecedent())
    consequent = list(rules[j].consequent())
    ruleItemset = str(antecedent + consequent)
    
    try:
        support = "{0:.2f}".format(freqItemsetsDict[ruleItemset] / transactions.count())
    except KeyError:
        next
    
    confidence = "{0:.2f}".format(rules[j].confidence())
    
    try:
        print(antecedent, '=>', consequent, '(support: '+ str(support), 'confidence: '+ str(confidence) + ')')
    except NameError:
        next

['21754', '22310', '84969', '22623', '22622', '48187"'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['22748'] (support: 0.20 confidence: 1.00)
['22749'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22749'] => ['22745'] (support: 0.20 confidence: 1.00)
['22749'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22749'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['21777'] (support: 0.20 confidence: 1.00)
['22749'] => ['22310'] (support: 0.20 confidence: 1.00)
['22749'] => ['22622'] (support: 0.20 confidence: 1.00)
['22749'] => ['22623'] (support: 0.20 confidence: 1.00)
['22749'] => ['21754'] (support: 0.20 confidence: 1.00)
['22749'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754',

['21754', '84969', '22622', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '48187"', '22745'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '48187"', '22745'] =>

['22622', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '

['21754', '22310', '21777', '22745'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['84969'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '21755'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '21755'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)


['21754', '22622', '21777', '22749', '22748', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22622', '21777', '22749', '22748', '22745', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22622', '21777', '22749', '22748', '22745', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22748', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22748', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22748', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22748', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22748', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22748', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622', '48187"', '"8487

...

['22622', '21777', '22748', '48187"', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22748', '48187"', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22748', '48187"', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22748', '48187"', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22748', '48187"', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22748', '48187"', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22748', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22748', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22748', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22748', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', 

['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '21777', '22749', '22748', '48187"'] => ['22622'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['22623'] (support: 0.20 confidence: 1.00)
['22622', '22749', '"84879', '21755'] => ['21754'] 

['21754', '22310', '22622', '21777', '48187"', '"84879', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22749', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '22745', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '22745', '21755'] => ['"84879'] (support: 0.20 confidence: 1.00)
[

['21754', '22310', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['22745'] (support: 0.20 confidence: 1.

['21777', '22749', '"84879', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['21777', '22749', '"84879', '21755'] => ['22623'] (support: 0.20 confidence: 1.00)
['21777', '22749', '"84879', '21755'] => ['21754'] (support: 0.20 confidence: 1.00)
['21777', '22749', '"84879', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['84969'] (support: 0.2

In [22]:
sc.stop()

Summary:

Output: ['21754', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)

Support: is the percentage of transactions in {Total transactions} that contain all three '21754', '22748' and '22745' together (20% of all orders had the three items together).

Confidence: is the percentage of transactions in {Total transactions} containing '21754', '22748' that also contains '22745'. 

The probability of having '22745', given that '21754', '22748' is already in the basket: 20% of orders having products '21754', '22748', also bought '22745'.

When the resulting confidence is smaller than minimum confidence, in this case .40, the association rule  ['21754', '22748'] => ['22745'] is dropped, otherwise it is added to the result.

So, the resulting confidence for this itemset  ['21754', '22748', '22745'] is 1 > .40, meaning that invoices with product '22745' also contained products '21754', '22748'.


